In [1]:
import torch
import jieba
from zhconv import convert
from torch.autograd import Variable
from utils.utils_sensation_lcsts import *
from seq2seq.sensation_scorer import SensationCNN
from utils.utils_sensation import input_txt_to_batch

# Run CNN Scorer

In [ ]:
train, dev, test, lang, max_q, max_r = prepare_data_seq(batch_size=128,
                                                        debug=False, shuffle=True,
                                                        pointer_gen=True,
                                                        output_vocab_size=50000,
                                                        thd=0.0)

opts = torch.load("save/sensation/512_0.9579935073852539/args.th")
sensation_model = SensationCNN(opts, lang)
checkpoint = torch.load("save/sensation/512_0.9579935073852539/sensation_scorer.th")
sensation_model.load_state_dict(checkpoint['model'])
sensation_model.cuda()

In [ ]:
yusan_data_path = 'dataset/yushan/eliminated_yushan_dataset.txt'
yusan_title = []
with open(yusan_data_path, 'r', encoding='utf8') as f:
     for line in f.readlines():
            if line != '\n':
                title, article = line.split('\t')
                yusan_title.append(convert(title, 'zh-cn'))
            
inputs = input_txt_to_batch(yusan_title, lang).cuda()
yusan_title_score = sensation_model(inputs).cpu().detach().numpy()
print(yusan_title_score.shape)

In [ ]:
# without score
results = ''
with open(yusan_data_path, 'r', encoding='utf8') as f:
     for i, line in enumerate(f.readlines()):
        if line != '\n':
            title, article = line.split('\t')
            processed_title = (" ".join(list(jieba.cut(convert(title, 'zh-cn')))))
            processed_article = (" ".join(list(jieba.cut(convert(article, 'zh-cn')))))
            results += (processed_title + '\t' + processed_article)

with open('dataset/yushan/yushan_processed_data_without_score.txt', 'w+', encoding='utf8') as f:
    f.write(results)

In [ ]:
# with score
results = ''
i = 0
with open(yusan_data_path, 'r', encoding='utf8') as f:
     for line in f.readlines():
        if line != '\n' and i < 324:
            title, article = line.split('\t')
            processed_title = (" ".join(list(jieba.cut(convert(title, 'zh-cn')))))
            processed_article = (" ".join(list(jieba.cut(convert(article, 'zh-cn')))))
            results += (processed_title + '\t' + str(yusan_title_score[i]) + '\t' + processed_article)
            i+=1
        
with open('dataset/yushan/yushan_processed_data_with_score.txt', 'w+', encoding='utf8') as f:
    f.write(results)

# Spliting Data

In [ ]:
data = []
with open('dataset/yushan/yushan_processed_data_with_score.txt', 'r', encoding='utf8') as f:
    for i in f.readlines():
        if i != '\n':
            data.append(i.strip())
len(data)

In [ ]:
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(data,test_size=0.05,random_state=42)
train_pairs, dev_pairs = train_test_split(train_pairs,test_size=0.075,random_state=42)
print(len(dev_pairs), len(test_pairs))

In [ ]:
train_data = '\n'.join(train_pairs)
with open('dataset/yushan/same_train.txt', 'w+', encoding='utf8') as f:
    f.write(train_data)

valid_data = '\n'.join(dev_pairs)
with open('dataset/yushan/same_dev.txt', 'w+', encoding='utf8') as f:
    f.write(valid_data)
    
test_data = '\n'.join(test_pairs)
with open('dataset/yushan/same_test.txt', 'w+', encoding='utf8') as f:
    f.write(test_data)

In [ ]:
data = []
with open('dataset/yushan/yushan_processed_data_without_score.txt', 'r', encoding='utf8') as f:
    for i in f.readlines():
        if i != '\n':
            data.append(i.strip())
len(data)

In [ ]:
from sklearn.model_selection import train_test_split
train_pairs, test_pairs = train_test_split(data,test_size=0.05,random_state=42)
train_pairs, dev_pairs = train_test_split(train_pairs,test_size=0.075,random_state=42)
len(test_pairs)

In [ ]:
train_data = '\n'.join(train_pairs)
with open('dataset/yushan/segment_train.txt', 'w+', encoding='utf8') as f:
    f.write(train_data)

valid_data = '\n'.join(dev_pairs)
with open('dataset/yushan/segment_valid.txt', 'w+', encoding='utf8') as f:
    f.write(valid_data)
    
test_data = '\n'.join(test_pairs)
with open('dataset/yushan/segment_test.txt', 'w+', encoding='utf8') as f:
    f.write(test_data)